<font color="#CC3D3D"><p>
# DNN Modeling with ktrain & seed ensemble

- ktrain 패키지는 주피터노트북을 restart하지 않고 loop로 반복해서 모델을 만들어 앙상블하면 원하는 성능이 나오지 않음
- 아래 코드는 이러한 문제를 해결한 것임.

**소스코드 저장**

In [35]:
import ktrain
from ktrain import tabular  # tabular data에 적용
import pandas as pd
import numpy as np

In [36]:
%%writefile ktrain_mlp.py
### Imports
import ktrain
from ktrain import tabular  # tabular data에 적용
import pandas as pd
import numpy as np

### Read data
# 1st round 1등팀 피처 사용
X_train = pd.read_csv('snd_sum_shap_0612.csv')
X_test = pd.read_csv('snd_sum_shap_te_0612.csv')
y_train = pd.read_csv('y_train.csv').group
IDtest = pd.read_csv('X_test.csv', encoding= 'cp949').custid.unique()
# 아래 tabular_from_df()에서는 target 필드까지 들어간 데이터프레임을 요구
train_df = pd.concat([X_train, pd.DataFrame(y_train)], axis=1)

### Preprocessing
# Scaling, Imputation, train/valid 분할 등의 전처리를 수행하고 나중에 사용할 trn, val, preproc를 반환.
# validation data의 비율(현재 20%)은 val_pct로 지정
trn, val, preproc = tabular.tabular_from_df(train_df, label_columns=['group'], val_pct=0.2, random_state=42)

### Modeling
# MLP를 사용하여 모델링 (tabular data는 MLP만 지원). 배치 사이즈는 성능에 지대한 영향을 미침.
model = tabular.tabular_classifier('mlp', trn)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=512)
# Model Fitting: 학습율(lr)과 에포크(epochs)가 성능에 지대한 영향을 미침.
hist = learner.fit_onecycle(lr=5e-3, epochs=3, checkpoint_folder='output')

### Prediction
predictor = ktrain.get_predictor(learner.model, preproc, batch_size=4096)
pred = predictor.predict(X_test, return_proba=True)    
t = pd.Timestamp.now()
fname = f"submissions/ktrain_mlp_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}{t.second:02}.csv"
pred = pd.DataFrame(pred)
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
submissions = pd.concat([pd.Series(IDtest, name="ID"), pred] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

Overwriting ktrain_mlp.py


**소스코드 실행**

In [37]:
import subprocess
from tqdm import tqdm

N = 10
for i in tqdm(range(N)):
    subprocess.run(['python', 'ktrain_mlp.py'])

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [47:54<00:00, 287.48s/it]


<font color="#CC3D3D"><p>
# End